# **Trade Data Analysis: Global Exports of Electrical Machinery (HS Code 85) (2016 - 2024)**

## Project Overview

This analysis examines electrical machinery export data from the UN Comtrade Database focusing on:
- Type: Goods (HS Code 85 - Electrical machinery and equipment)
- Period: 2016-2024
- Trade Flow: Export
- Coverage: All countries
- Partner: World
- Measure: USD

In [1]:
# @title Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
# plt.style.use('seaborn') # This line is causing the error. Remove or replace it.
# Instead of 'seaborn', use one of Seaborn's built-in styles:
sns.set_theme(style="darkgrid") # or "whitegrid", "dark", "white", "ticks"
# This will set the Seaborn style, which also affects Matplotlib plots.
sns.set_palette("husl")

## Task 1: Data Preparation

### 1. Data Import and Cleaning
- **Data Source**: UN Comtrade Database electrical machinery exports
- **Data Format**: CSV with trade flow information
- **Time Period**: 2016-2024 comprehensive data

### 2. Data Processing
- **Filtering**: Select relevant trade flows and partners
- **Value Processing**: Handle primary and FOB values
- **Quality Control**: Remove incomplete or invalid entries

### 3. Export Value Analysis
- **Value Threshold**: Filter countries with exports > $500M
- **Data Validation**: Ensure data completeness
- **Output Format**: Structured CSV export

In [2]:
# @title Part 1 – Data Preparation : Import, Clean, and Filter Export Data for Analysis
def load_and_clean_data():
    """
    Load and clean the UN Comtrade data
    """
    # Load data from GitHub URL
    url = 'https://raw.githubusercontent.com/arunvithyasegar/UN_Trade_dataset/main/TradeData_5_10_2025_12_27_1.csv'
    df = pd.read_csv(url, encoding='cp1252')

    # Use primaryValue as main value column, fallback to fobvalue
    df['export_value'] = df['primaryValue'].fillna(df['fobvalue'])

    # Filter for World partners and Export trade flow
    df = df[(df['partnerDesc'] == 'World') & (df['flowDesc'] == 'Export')]

    # Keep essential columns
    clean_df = df[['refYear', 'reporterDesc', 'export_value']]

    # Filter for countries with export values above $500 million in 2024
    df_2024 = clean_df[clean_df['refYear'] == 2024]
    high_export_countries = df_2024[df_2024['export_value'] >= 500000000]['reporterDesc'].unique()

    # Filter dataset for these countries
    filtered_df = clean_df[clean_df['reporterDesc'].isin(high_export_countries)]

    return filtered_df

# Load and clean the data
filtered_df = load_and_clean_data()

# Display first few rows and basic information
print("Dataset Overview:")
display(filtered_df.head())
print("\nDataset Info:")
display(filtered_df.info())

# Save cleaned dataset
filtered_df.to_csv('cleaned_trade_data.csv', index=False)
print("\nCleaned dataset saved as 'cleaned_trade_data.csv'")

Dataset Overview:


,refYear,reporterDesc,export_value
7,2016,Australia,2.672953e+09
11,2016,Armenia,1.334843e+07
13,2016,Belgium,9.699999e+09
16,2016,Bosnia Herzegovina,2.393363e+08
18,2016,Brazil,3.226944e+09



Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 441 entries, 7 to 1401
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   refYear       441 non-null    int64  
 1   reporterDesc  441 non-null    object 
 2   export_value  441 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 13.8+ KB


None


Cleaned dataset saved as 'cleaned_trade_data.csv'


## Task 2: Growth Trend Analysis

### 1. Growth Rate Computation
- **YoY Growth**: Calculate year-on-year growth rates
- **Average Growth**: Determine mean annual growth rates
- **Trend Analysis**: Identify consistent patterns

### 2. Performance Rankings
- **Top Performers**: Identify fastest growing exporters
- **Growth Stability**: Assess consistency of growth
- **Market Position**: Evaluate relative performance

### 3. Visualization and Trends
- **Growth Trajectories**: Plot individual country trends
- **Comparative Analysis**: Benchmark against global average
- **Market Dynamics**: Identify emerging patterns

In [3]:
# Create pivot table for analysis
pivot_df = filtered_df.pivot(index='reporterDesc', columns='refYear', values='export_value')
pivot_df.columns = [str(col) for col in pivot_df.columns]

# Calculate YoY growth rates
growth_df = pd.DataFrame(index=pivot_df.index)
for year in range(2017, 2025):
    current_year = str(year)
    prev_year = str(year - 1)
    growth_df[f'growth_{current_year}'] = (pivot_df[current_year] - pivot_df[prev_year]) / pivot_df[prev_year]

# Calculate average annual growth rate
growth_df['avg_growth'] = growth_df.mean(axis=1)

# Display top 10 countries by average growth
print("Top 10 Countries by Average Growth Rate:")
top_growth = growth_df['avg_growth'].sort_values(ascending=False).head(10)
display(top_growth.apply(lambda x: f"{x:.2%}"))

# Visualize export trends for top 3 countries
top_3_countries = growth_df.nlargest(3, 'avg_growth').index

# Create line plot using Plotly
fig = go.Figure()

for country in top_3_countries:
    fig.add_trace(go.Scatter(
        x=pivot_df.columns,
        y=pivot_df.loc[country] / 1e9,  # Convert to billions
        mode='lines+markers',
        name=country
    ))

fig.update_layout(
    title='Export Trends for Top 3 Growth Countries (2016-2024)',
    xaxis_title='Year',
    yaxis_title='Export Value (Billion USD)',
    legend_title='Country',
    hovermode='x unified',
    showlegend=True,
    height=600
)

fig.show()

# Comparison with global average
global_avg = pivot_df.mean() / 1e9  # Convert to billions
global_avg_df = pd.DataFrame({
    'Year': global_avg.index,
    'Global Average': global_avg.values
})

fig = go.Figure()

# Add global average
fig.add_trace(go.Scatter(
    x=global_avg_df['Year'],
    y=global_avg_df['Global Average'],
    mode='lines+markers',
    name='Global Average',
    line=dict(color='black', width=3, dash='dash')
))

# Add top 3 countries
for country in top_3_countries:
    fig.add_trace(go.Scatter(
        x=pivot_df.columns,
        y=pivot_df.loc[country] / 1e9,
        mode='lines+markers',
        name=country
    ))

fig.update_layout(
    title='Export Trends: Top 3 Countries vs. Global Average (2016-2024)',
    xaxis_title='Year',
    yaxis_title='Export Value (Billion USD)',
    legend_title='Country',
    hovermode='x unified',
    height=600
)

fig.show()

Top 10 Countries by Average Growth Rate:


,avg_growth
reporterDesc,
Armenia,273.80%
India,22.79%
Bosnia Herzegovina,17.55%
North Macedonia,15.72%
Rep. of Moldova,12.87%
Greece,11.61%
Dominican Rep.,10.20%
Malaysia,10.16%
Croatia,10.03%


## Task 3: Volatility & Classification Analysis

### 1. Volatility Assessment
- **Metric Calculation**: Compute coefficient of variation
- **Volatility Ranking**: Identify most volatile markets
- **Stability Analysis**: Evaluate export consistency

### 2. Market Classification
- **Segmentation Criteria**: Define growth-volatility quadrants
- **Category Assignment**: Classify countries by performance
- **Segment Analysis**: Evaluate group characteristics

### 3. Pattern Visualization
- **Quadrant Analysis**: Plot growth-volatility relationships
- **Distribution Analysis**: Examine segment compositions
- **Market Structure**: Reveal underlying patterns

In [4]:
# @title Part 3 – Volatility & Exporter Classification : Analyze Export Volatility and Classify Countries by Growth & Stability
# Calculate volatility (coefficient of variation)
volatility = pivot_df.std(axis=1) / pivot_df.mean(axis=1)
growth_df['volatility'] = volatility

# Display top 10 most volatile exporters
print("Top 10 Most Volatile Exporters:")
display(volatility.sort_values(ascending=False).head(10))

# Plot volatility for top 10 using Plotly
top_10_volatile = volatility.nlargest(10)
fig = px.bar(
    x=top_10_volatile.index,
    y=top_10_volatile.values,
    labels={'x': 'Country', 'y': 'Volatility (Coefficient of Variation)'},
    title='Top 10 Most Volatile Exporters',
    color=top_10_volatile.values,
    color_continuous_scale='Reds'
)

fig.update_layout(height=600)
fig.show()

# Classify countries based on growth and volatility
median_growth = growth_df['avg_growth'].median()
median_volatility = growth_df['volatility'].median()

conditions = [
    (growth_df['avg_growth'] >= median_growth) & (growth_df['volatility'] < median_volatility),
    (growth_df['avg_growth'] >= median_growth) & (growth_df['volatility'] >= median_volatility),
    (growth_df['avg_growth'] < median_growth) & (growth_df['volatility'] < median_volatility),
    (growth_df['avg_growth'] < median_growth) & (growth_df['volatility'] >= median_volatility)
]

choices = ['Stable High-Growth', 'Volatile High-Growth', 'Stable Low-Growth', 'Volatile Low-Growth']
growth_df['classification'] = np.select(conditions, choices, default='Unknown')

# Create quadrant plot using Plotly
fig = px.scatter(
    x=growth_df['volatility'],
    y=growth_df['avg_growth'] * 100,  # Convert to percentage
    hover_name=growth_df.index,
    labels={
        'x': 'Volatility (Coefficient of Variation)',
        'y': 'Average Annual Growth Rate (%)'
    },
    title='Volatility vs. Average Growth Rate',
    color=growth_df['classification'],
    color_discrete_map={
        'Stable High-Growth': 'green',
        'Volatile High-Growth': 'orange',
        'Stable Low-Growth': 'blue',
        'Volatile Low-Growth': 'red'
    },
    size=pivot_df['2024'] / 1e9,  # Size by export value in 2024
    size_max=50
)

# Add quadrant lines
fig.add_hline(y=median_growth * 100, line_dash="dash", line_width=1)
fig.add_vline(x=median_volatility, line_dash="dash", line_width=1)

# Add quadrant labels
fig.add_annotation(x=median_volatility/2, y=median_growth * 150,
                  text="Stable High-Growth", showarrow=False)
fig.add_annotation(x=median_volatility * 1.5, y=median_growth * 150,
                  text="Volatile High-Growth", showarrow=False)
fig.add_annotation(x=median_volatility/2, y=median_growth * 50,
                  text="Stable Low-Growth", showarrow=False)
fig.add_annotation(x=median_volatility * 1.5, y=median_growth * 50,
                  text="Volatile Low-Growth", showarrow=False)

fig.update_layout(height=700)
fig.show()

# Create pie chart for classification distribution
classification_counts = growth_df['classification'].value_counts()
fig = px.pie(
    values=classification_counts.values,
    names=classification_counts.index,
    title='Distribution of Countries by Classification',
    color=classification_counts.index,
    color_discrete_map={
        'Stable High-Growth': 'green',
        'Volatile High-Growth': 'orange',
        'Stable Low-Growth': 'blue',
        'Volatile Low-Growth': 'red'
    },
    hole=0.4
)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(height=500)
fig.show()

Top 10 Most Volatile Exporters:


,0
reporterDesc,
Armenia,1.489624
India,0.567506
Bosnia Herzegovina,0.384154
Greece,0.351884
North Macedonia,0.313165
Poland,0.302588
Croatia,0.275009
Egypt,0.270384
Malaysia,0.260595


## Task 4: Statistical & Forecasting Analysis

### 1. Growth Rate Distribution Analysis
- **Normal Distribution Fit**: Analyzing the statistical distribution of growth rates
- **Statistical Measures**: Key metrics including mean, median, and standard deviation
- **Performance Segments**: Categorization of countries by growth performance

### 2. Performance Analysis
- **Top/Bottom Performers**: Identifying leading and lagging countries
- **Growth Rate Segments**: Analysis of distinct growth segments
- **Regional Patterns**: Geographical trends and clusters

### 3. Key Insights
- **Distribution Characteristics**: Shape, spread, and central tendency
- **Growth Patterns**: Temporal and structural patterns
- **Market Implications**: Impact on trade dynamics and competitiveness

In [5]:
# @title Statistical & Forecasting Analysis : Explore Growth Distribution, Identify Extremes, and Visualize Patterns
# Statistical & Forecasting Analysis

# 1. Growth Rate Distribution Analysis
growth_mean = growth_df['avg_growth'].mean()
growth_std = growth_df['avg_growth'].std()

# Create performance segments
growth_df['performance_segment'] = pd.qcut(growth_df['avg_growth'],
                                         q=4,
                                         labels=['Low Growth', 'Moderate Growth',
                                                'High Growth', 'Very High Growth'])

# Distribution Plot with Enhanced Features
fig = go.Figure()

# Add histogram with proper percentage scaling
fig.add_trace(go.Histogram(
    x=growth_df['avg_growth'] * 100,
    name='Growth Distribution',
    nbinsx=30,
    histnorm='probability density',
    marker_color='rgba(73, 160, 181, 0.7)',
    hovertemplate='Growth Rate: %{x:.1f}%<br>Density: %{y:.3f}<extra></extra>'
))

# Add normal distribution curve
x_range = np.linspace(growth_df['avg_growth'].min(), growth_df['avg_growth'].max(), 100)
y_normal = stats.norm.pdf(x_range, growth_mean, growth_std)
hist_values, _ = np.histogram(growth_df['avg_growth'] * 100, bins=30, density=True)
scaling_factor = max(hist_values) / max(y_normal)
y_normal = y_normal * scaling_factor

fig.add_trace(go.Scatter(
    x=x_range * 100,
    y=y_normal,
    mode='lines',
    name='Normal Distribution',
    line=dict(color='red', width=2),
    hovertemplate='Growth Rate: %{x:.1f}%<br>Density: %{y:.3f}<extra></extra>'
))

# Add reference lines and annotations
fig.add_vline(x=growth_mean * 100,
              line_dash='dash',
              line_color='green',
              annotation_text=f'Mean: {growth_mean:.1%}')
fig.add_vline(x=(growth_mean + growth_std) * 100,
              line_dash='dot',
              line_color='gray',
              annotation_text='+1 SD')
fig.add_vline(x=(growth_mean - growth_std) * 100,
              line_dash='dot',
              line_color='gray',
              annotation_text='-1 SD')

fig.update_layout(
    title={
        'text': 'Distribution of Average Annual Growth Rates',
        'font': dict(size=20)
    },
    xaxis_title='Average Annual Growth Rate (%)',
    yaxis_title='Density',
    height=600,
    showlegend=True,
    hovermode='x unified',
    xaxis=dict(
        tickformat='.1f',
        gridcolor='rgba(0,0,0,0.1)',
        zerolinecolor='black',
        zerolinewidth=2
    ),
    yaxis=dict(
        gridcolor='rgba(0,0,0,0.1)',
        zerolinecolor='black',
        zerolinewidth=2
    ),
    plot_bgcolor='white'
)

fig.show()

# 2. Performance Analysis

# Create performance summary table
performance_summary = pd.DataFrame({
    'Segment': growth_df['performance_segment'].value_counts().index,
    'Count': growth_df['performance_segment'].value_counts().values,
    'Avg Growth': growth_df.groupby('performance_segment')['avg_growth'].mean(),
    'Min Growth': growth_df.groupby('performance_segment')['avg_growth'].min(),
    'Max Growth': growth_df.groupby('performance_segment')['avg_growth'].max(),
    'Avg Volatility': growth_df.groupby('performance_segment')['volatility'].mean()
})

# Format percentage columns
for col in ['Avg Growth', 'Min Growth', 'Max Growth']:
    performance_summary[col] = performance_summary[col].apply(lambda x: f'{x:.1%}')
performance_summary['Avg Volatility'] = performance_summary['Avg Volatility'].apply(lambda x: f'{x:.2f}')

print('Performance Segment Analysis:')
display(performance_summary)

# Create box plot for growth rate segments
fig = go.Figure()

for segment in growth_df['performance_segment'].unique():
    segment_data = growth_df[growth_df['performance_segment'] == segment]['avg_growth'] * 100
    fig.add_trace(go.Box(
        y=segment_data,
        name=segment,
        boxpoints='all',
        jitter=0.3,
        pointpos=-1.8,
        marker_color='rgba(73, 160, 181, 0.7)',
        line_color='rgb(8, 81, 156)'
    ))

fig.update_layout(
    title='Growth Rate Distribution by Performance Segment',
    yaxis_title='Average Annual Growth Rate (%)',
    height=500,
    showlegend=False,
    boxmode='group'
)

fig.show()

# 3. Key Statistical Insights
print('\nKey Statistical Insights:')
print('-' * 50)

# Normality test
stat, p_value = stats.normaltest(growth_df['avg_growth'])
normality_status = 'normally' if p_value > 0.05 else 'not normally'
print(f'1. Growth rates are {normality_status} distributed (p-value: {p_value:.3f})')

# @title Statistical & Forecasting Analysis : Explore Growth Distribution, Identify Extremes, and Visualize Patterns
# Statistical & Forecasting Analysis

# 1. Growth Rate Distribution Analysis
growth_mean = growth_df['avg_growth'].mean()
growth_std = growth_df['avg_growth'].std()

# Create performance segments
growth_df['performance_segment'] = pd.qcut(growth_df['avg_growth'],
                                         q=4,
                                         labels=['Low Growth', 'Moderate Growth',
                                                'High Growth', 'Very High Growth'])

# Distribution Plot with Enhanced Features
fig = go.Figure()

# Add histogram with proper percentage scaling
fig.add_trace(go.Histogram(
    x=growth_df['avg_growth'] * 100,
    name='Growth Distribution',
    nbinsx=30,
    histnorm='probability density',
    marker_color='rgba(73, 160, 181, 0.7)',
    hovertemplate='Growth Rate: %{x:.1f}%<br>Density: %{y:.3f}<extra></extra>'
))

# Add normal distribution curve
x_range = np.linspace(growth_df['avg_growth'].min(), growth_df['avg_growth'].max(), 100)
y_normal = stats.norm.pdf(x_range, growth_mean, growth_std)
hist_values, _ = np.histogram(growth_df['avg_growth'] * 100, bins=30, density=True)
scaling_factor = max(hist_values) / max(y_normal)
y_normal = y_normal * scaling_factor

fig.add_trace(go.Scatter(
    x=x_range * 100,
    y=y_normal,
    mode='lines',
    name='Normal Distribution',
    line=dict(color='red', width=2),
    hovertemplate='Growth Rate: %{x:.1f}%<br>Density: %{y:.3f}<extra></extra>'
))

# Add reference lines and annotations
fig.add_vline(x=growth_mean * 100,
              line_dash='dash',
              line_color='green',
              annotation_text=f'Mean: {growth_mean:.1%}')
fig.add_vline(x=(growth_mean + growth_std) * 100,
              line_dash='dot',
              line_color='gray',
              annotation_text='+1 SD')
fig.add_vline(x=(growth_mean - growth_std) * 100,
              line_dash='dot',
              line_color='gray',
              annotation_text='-1 SD')

fig.update_layout(
    title={
        'text': 'Distribution of Average Annual Growth Rates',
        'font': dict(size=20)
    },
    xaxis_title='Average Annual Growth Rate (%)',
    yaxis_title='Density',
    height=600,
    showlegend=True,
    hovermode='x unified',
    xaxis=dict(
        tickformat='.1f',
        gridcolor='rgba(0,0,0,0.1)',
        zerolinecolor='black',
        zerolinewidth=2
    ),
    yaxis=dict(
        gridcolor='rgba(0,0,0,0.1)',
        zerolinecolor='black',
        zerolinewidth=2
    ),
    plot_bgcolor='white'
)

fig.show()

# 2. Performance Analysis

# Create performance summary table
performance_summary = pd.DataFrame({
    'Segment': growth_df['performance_segment'].value_counts().index,
    'Count': growth_df['performance_segment'].value_counts().values,
    'Avg Growth': growth_df.groupby('performance_segment')['avg_growth'].mean(),
    'Min Growth': growth_df.groupby('performance_segment')['avg_growth'].min(),
    'Max Growth': growth_df.groupby('performance_segment')['avg_growth'].max(),
    'Avg Volatility': growth_df.groupby('performance_segment')['volatility'].mean()
})

# Format percentage columns
for col in ['Avg Growth', 'Min Growth', 'Max Growth']:
    performance_summary[col] = performance_summary[col].apply(lambda x: f'{x:.1%}')
performance_summary['Avg Volatility'] = performance_summary['Avg Volatility'].apply(lambda x: f'{x:.2f}')

print('Performance Segment Analysis:')
display(performance_summary)

# Create box plot for growth rate segments
fig = go.Figure()

for segment in growth_df['performance_segment'].unique():
    segment_data = growth_df[growth_df['performance_segment'] == segment]['avg_growth'] * 100
    fig.add_trace(go.Box(
        y=segment_data,
        name=segment,
        boxpoints='all',
        jitter=0.3,
        pointpos=-1.8,
        marker_color='rgba(73, 160, 181, 0.7)',
        line_color='rgb(8, 81, 156)'
    ))

fig.update_layout(
    title='Growth Rate Distribution by Performance Segment',
    yaxis_title='Average Annual Growth Rate (%)',
    height=500,
    showlegend=False,
    boxmode='group'
)

fig.show()

# 3. Key Statistical Insights
print('\nKey Statistical Insights:')
print('-' * 50)

# Normality test
stat, p_value = stats.normaltest(growth_df['avg_growth'])
normality_status = 'normally' if p_value > 0.05 else 'not normally'
print(f'1. Growth rates are {normality_status} distributed (p-value: {p_value:.3f})')

# Distribution characteristics
print(f'2. Mean growth rate: {growth_mean:.1%} with standard deviation: {growth_std:.1%}')
print(f'3. Skewness: {growth_df["avg_growth"].skew():.2f} ({"positive" if growth_df["avg_growth"].skew() > 0 else "negative"} skew)')
print(f'4. Kurtosis: {growth_df["avg_growth"].kurtosis():.2f} ({"heavy" if growth_df["avg_growth"].kurtosis() > 3 else "light"} tails)')

# Top performers
top_performers = growth_df.nlargest(3, 'avg_growth')
print('\nTop 3 Performers:')
for idx, row in top_performers.iterrows():
    print(f'- {idx}: {row["avg_growth"]:.1%} growth with {row["volatility"]:.2f} volatility')
# Top performers
top_performers = growth_df.nlargest(3, 'avg_growth')
print('\nTop 3 Performers:')
for idx, row in top_performers.iterrows():
    print(f'- {idx}: {row["avg_growth"]:.1%} growth with {row["volatility"]:.2f} volatility')

Performance Segment Analysis:


,Segment,Count,Avg Growth,Min Growth,Max Growth,Avg Volatility
performance_segment,,,,,,
Low Growth,Low Growth,13,2.4%,0.1%,4.4%,0.09
Moderate Growth,Moderate Growth,12,5.4%,4.5%,6.1%,0.15
High Growth,High Growth,12,7.3%,6.2%,8.7%,0.20
Very High Growth,Very High Growth,12,34.5%,9.6%,273.8%,0.41



Key Statistical Insights:
--------------------------------------------------
1. Growth rates are not normally distributed (p-value: 0.000)


Performance Segment Analysis:


,Segment,Count,Avg Growth,Min Growth,Max Growth,Avg Volatility
performance_segment,,,,,,
Low Growth,Low Growth,13,2.4%,0.1%,4.4%,0.09
Moderate Growth,Moderate Growth,12,5.4%,4.5%,6.1%,0.15
High Growth,High Growth,12,7.3%,6.2%,8.7%,0.20
Very High Growth,Very High Growth,12,34.5%,9.6%,273.8%,0.41



Key Statistical Insights:
--------------------------------------------------
1. Growth rates are not normally distributed (p-value: 0.000)
2. Mean growth rate: 12.2% with standard deviation: 38.4%
3. Skewness: 6.87 (positive skew)
4. Kurtosis: 47.71 (heavy tails)

Top 3 Performers:
- Armenia: 273.8% growth with 1.49 volatility
- India: 22.8% growth with 0.57 volatility
- Bosnia Herzegovina: 17.5% growth with 0.38 volatility

Top 3 Performers:
- Armenia: 273.8% growth with 1.49 volatility
- India: 22.8% growth with 0.57 volatility
- Bosnia Herzegovina: 17.5% growth with 0.38 volatility
